<a href="https://colab.research.google.com/github/hirossk/0809python/blob/main/0809LLM%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%81%BE%E3%81%97%E3%82%87%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIによる判定

学習済みモデルのニューラルネットワークを使って画像を読み込んで解析してみましょう。

In [8]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing

In [9]:
import gradio as gr
import requests
import torch
import torch.nn as nn
from PIL import Image
from torchvision.models import resnet50
from torchvision.transforms import functional as F


In [ ]:
def main():
    # モデルの準備
    model = resnet50(pretrained=True)
    model.eval()

    # ImageNetのラベルの取得
    response = requests.get("https://git.io/JJkYN")
    labels = response.text.split("\n")

    # 画像分類を行う関数を定義
    @torch.no_grad()
    def inference(input_img):
        print("inference")
        img = Image.fromarray(input_img.astype("uint8"), "RGB")
        print("test")

                # # 前処理
        img = F.resize(img, (224, 224))
        img = F.to_tensor(img)
        img = img.unsqueeze(0)
        img = F.normalize(
            img,
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )

        # # 推論
        output = model(img).squeeze(0)
        probs = nn.functional.softmax(output, dim=0).numpy()

        # # ラベルごとの確率をdictとして返す
        return {labels[i]: float(probs[i]) for i in range(1000)}
        # return sepia_img

    # 入力の形式を画像とする
    inputs = gr.Image(label="Image")
    # 出力はラベル形式で，top5まで表示する
    outputs = gr.Label(num_top_classes=10)
    # サーバーの立ち上げ
    interface = gr.Interface(inference, gr.Image(), outputs=outputs)
    interface.launch()

main()

In [ ]:
!pip install transformers accelerate


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("cyberagent/open-calm-7b", device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("cyberagent/open-calm-7b")

inputs = tokenizer("AIによって私達の暮らしは、", return_tensors="pt").to(model.device)
with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)


config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/42.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

AIによって私達の暮らしは、もっと便利で快適で豊かなものになるはずです。
しかし、ITやIoT技術が発達しても、それはあくまでも道具に過ぎません。そして、その道具を使いこなすのは「人」です。人が持つ感性・感受性をいかに磨き上げるのか。それがこれからの企業にとって、一番重要な戦略ではないでしょうか?そのために、私達は今まで以上に人と


In [7]:
inputs = tokenizer("AIによって私達の暮らしは、", return_tensors="pt").to(model.device)
with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)


AIによって私達の暮らしは、ますます便利で快適になってきました。そして今、人工知能を搭載したロボットが身近になりつつあるのをご存知ですか?ここでは身近な生活の中にある、主に製造業などの産業用途としての人工知能の活用事例をご紹介します。.  Pythonで機械学習をしてみたい方向けにPythonとNumPyの基礎から実践まで解説した記事です。データセットを用意する初期設定や分類・回帰・クラスタリングなど、具体的なプログラミング手法に関して説明しています。また、最後に簡単な予測問題や行列分解についても紹介しています。.
Phone:(994) 726-7763 x 8622


In [1]:
!pip install transformers
!pip install torch torchvision torchaudio
!pip install accelerate
!git clone https://huggingface.co/cyberagent/open-calm-3b


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "cyberagent/open-calm-3b"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# モデルをCPUまたはGPUに明示的にロードするようにautoから変更
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

model.eval()

requests = [
    "「馬が合う」はどう言う意味ですか",
    "How to make an authentic Spanish Omelette?",
]

system_message = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {user_input} ASSISTANT:"

for req in requests:
    input_req = system_message.format(user_input=req)
    input_ids = tokenizer.encode(input_req, return_tensors="pt").to(device)
    tokens = model.generate(
        input_ids,
        max_new_tokens=1024,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    out = tokenizer.decode(tokens[0][len(input_ids[0]):], skip_special_tokens=True)
    print("USER:\n" + req)
    print("ASSISTANT:\n" + out)
    print()
    print()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


USER:
「馬が合う」はどう言う意味ですか
ASSISTANT:
 So I told me, I feel like everything I had or have.
また、このアプリを使って、1週間前に買ったばかりの「1Password」のアプリや、最近買ったばかりの無料アプリなども、すでにロックされてしまったものもありますが、ロックが解除されているものもいくつかあるので、ロックされるのを待つ、というのが1つ解決策かもしれません。. 今日一日の歩数は、87760歩。 今日も、1年365日を、6月23日から365日とすると、今日が1年の6月23日、ということで、計算してみました。 今日は、この計算が正しいか、確かめたいので、計算してみました。 1年のカレンダー。 2020.1.1. 2019/04/26 · 「1Password」とは、パスワードや口座番号、クレジットカード番号など、重要情報をひとつのアプリで確認、保存し、安全なインターネットバンキングなどを楽しめるアプリです。 iOS版「1Password」は、iPhoneやiPadなど、以下のOSに対応しています。. 今日(4月23日)の歩数は、82563歩。 今日は、この 計算が正しいか、確かめたいので、計算 してみました。 今年の 1 年カレンダーの 1 日にあたっているので、この計算が、正しいか、確認したいと思います。 計算結果は、この表に書いてあります。2019. 今日の運勢(2020年) 明日の恋愛運(2020年) 今日の運勢(2020年) 今日1日をいい1日にするために朝占いがおすすめ。今日の運勢の集計データや、星座別の占いを毎日更新。各星座ごとの毎日の占いに加えて、月ごとの毎日占いが閲覧できます。.


USER:
How to make an authentic Spanish Omelette?
ASSISTANT:
 Getting a SPAN CERTIFIED inspection in your homestay inspection are going to work hard! Describe the SPAN CERTIFIED inspection philes and quasi inspection phones at your local or in KMN. H

In [3]:
requests = [
    "北海道の特徴はなんですか？",
]

system_message = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {user_input} ASSISTANT:"

for req in requests:
    input_req = system_message.format(user_input=req)
    input_ids = tokenizer.encode(input_req, return_tensors="pt").to(device)
    tokens = model.generate(
        input_ids,
        max_new_tokens=1024,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    out = tokenizer.decode(tokens[0][len(input_ids[0]):], skip_special_tokens=True)
    print("USER:\n" + req)
    print("ASSISTANT:\n" + out)
    print()
    print()

USER:
北海道の特徴はなんですか？
ASSISTANT:
 北海道の人口は約540万人、面積は日本の本州1.5倍で広大で道内9つの市や町から成っています。 気候は亜寒帯湿潤気候に属し、年間を通して高温多湿な気候です。 主な都市:札幌(人口約530万人、面積4,422.77平方キロメートル) 気候:亜寒帯湿潤気候、北海道の特徴のひとつであり、最も寒冷な時期と最も温暖な時期がほぼ同じである。 主な経済都市:函館市(人口約25.7万人、面積約2.6平方キロメートル) 主要産業:漁業、林業、化学工業、鉱業、機械製造業、食料品製造業。 主な産業:農業、畜産業、製造業、林業、漁業、卸売業、小売業。 主な産業施設:石油産業 都市機能、産業力とも比較的低いです。 また道北、道央、道東や道南など地域にも大きな特徴があります。 気候:亜寒帯湿潤気候に属し、年間を通して高温多湿な気候。 気温、降水量:札幌は北海道の中心であり、全国平均より若干温暖である。 降雪量は北海道では少ない。 夏や梅雨も北海道としては暑くない。 主なイベント:さっぽろ雪まつり、さっぽろライラックまつり、さっぽろ大通ビアガーデン、さっぽろホワイトイルミネーション、さっぽろ雪まつり大通り会場、さっぽろ雪まつり大通公園、さっぽろ大通ふれあい広場、さっぽろホワイトイルミネーション、さっぽろ雪まつり大通公園、日本さくら名所100選、さっぽろ大通ビアガーデン、さっぽろライラックまつり、さっぽろ大通公園・札幌大通ビアガーデンコンサート、さっぽろホワイトイルミネーション・さっぽろテレビ塔、さっぽろ雪まつり大通り会場、さっぽろ雪まつり大通公園、さっぽろ大通ふれあい広場でイベント、さっぽろ雪まつり大通公園、北海道大沼国定公園、北海道遺産、北海道遺産選定、函館冬季道路使用可能証明書発行証明書等発行件数、登録件数はさっぽろ雪まつり、さっぽろ大通スキー場開設・供用開始件数は北海道と都道府県で割合(H22. 3.14現在) 北海道と北海道観光振興機構による「平成22年度北海道における旅行先別宿泊旅行者動向調査」による。 主なイベント・行事:さっぽろ雪まつり、さっぽろ大通ビアガーデン、さっぽろ雪まつり大通り会場、さっぽろホワイトイルミネーション、さっぽろ雪まつり大通公園、さっぽろテレビ塔、さっぽろ雪まつり開催期間中の雪道運転講習、さっぽろ雪